<a href="https://colab.research.google.com/github/tienhuynh96/NLP_Projects/blob/main/Demo_POS_Penn_BERT_Pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install evaluate accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 35.9 MB/s eta 0:00:00


In [2]:
# Importing List type from Typing module
from typing import List

# Importing numpy library
import numpy as np

# Importing torch library for tensor
import torch

# Importing evaluate library
import evaluate

# Import train_test_split function from sklearn's model_selection module
from sklearn.model_selection import train_test_split

In [3]:
# Importing the Natural Language Toolkit (NLTK) library
import nltk

# Downloading the 'treebank' dataset provided by NLTK - a popular corpus of parsed and tagged sentences
nltk.download('treebank')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

## 1. Load pre-trained checkpoints

In [4]:
# Importing class AutoTokenizer, AutoModelForTokenClassification from transformer library
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Defining the name of the pre_trained model to be used
model_name = "QCRI/bert-base-multilingual-cased-pos-english"

# Creating a tokenizer object by loading the tokenizer associated with the pre_trained model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Creating a model object by loading from pretrained model
model = AutoModelForTokenClassification.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of the model checkpoint at QCRI/bert-base-multilingual-cased-pos-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

## 2. Data Preparation

### 2.1 Load data from nltk corpus

In [6]:
# Retrieve tagged sentences from the Treebank corpus provided by NLTK
tagged_sentences = nltk.corpus.treebank.tagged_sents()
print('Number of samples:', len(tagged_sentences))
print(tagged_sentences[0])


Number of samples: 3914
[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]


In [7]:
# Initialize empty lists of sentences and sentence_tags
sentences, sentence_tags = [], []

# Iterate through each tagged sentence in the tagged_sentences list
for tagged_sentence in tagged_sentences:

  # Unzip the (word, tag) tuples into separate lists for words and tags
  sentence, tags = zip(*tagged_sentence)

  # Convert words to lowercase and append to the sentences list
  sentences.append([word.lower() for word in sentence])

  # Append the tags to the sentence_tags list
  sentence_tags.append([tag for tag in tags])
  # sentences.append(sentence)
  # sentence_tags.append(tags)

In [32]:
print(sentence_tags[0])
print(sentences[0])


['NNP', 'NNP', ',', 'CD', 'NNS', 'JJ', ',', 'MD', 'VB', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', 'NNP', 'CD', '.']
['pierre', 'vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov.', '29', '.']


### 2.2 Load dictionary to encode label from model config

In [9]:
# Importing defaultdict from collections module
from collections import defaultdict

# Creating a defaultdic object with integer type, label to id in model.
label2id = defaultdict(int, model.config.label2id)
# Creating a dictionary named id2label where keys are values from label2id and values are corresponding keys from label2id
id2label = {id: label for label, id in label2id.items()}

In [10]:
model.config.label2id
id2label

{7: '#',
 6: '$',
 5: "''",
 2: ',',
 17: '-LRB-',
 32: '-RRB-',
 4: '.',
 3: ':',
 8: 'CC',
 9: 'CD',
 10: 'DT',
 11: 'EX',
 12: 'FW',
 13: 'IN',
 14: 'JJ',
 15: 'JJR',
 16: 'JJS',
 18: 'LS',
 19: 'MD',
 20: 'NN',
 21: 'NNP',
 22: 'NNPS',
 23: 'NNS',
 0: 'O',
 24: 'PDT',
 25: 'POS',
 26: 'PRP',
 27: 'PRP$',
 28: 'RB',
 29: 'RBR',
 30: 'RBS',
 31: 'RP',
 33: 'SYM',
 34: 'TO',
 35: 'UH',
 36: 'VB',
 37: 'VBD',
 38: 'VBG',
 39: 'VBN',
 40: 'VBP',
 41: 'VBZ',
 42: 'WDT',
 43: 'WP',
 44: 'WP$',
 45: 'WRB',
 1: '``'}

In [11]:
len(label2id)

46

###2.3 Split dataset into train, val and test set

In [12]:
# Split train set: 70%, test set: 30%
train_sentences, test_sentences, train_tags, test_tags = train_test_split(
    sentences,
    sentence_tags,
    test_size = 0.3
)

In [13]:
# Split test set: 15%, valid set: 15%
valid_sentences, test_sentences, valid_tags, test_tags = train_test_split(
    test_sentences,
    test_tags,
    test_size = 0.5
)

In [14]:
train_tags[0]

['IN',
 'CD',
 'NN',
 ',',
 'NNP',
 'VBZ',
 'VBN',
 '-NONE-',
 'TO',
 'VB',
 'VBN',
 'IN',
 'RB',
 'JJ',
 'IN',
 'CD',
 'NN',
 'IN',
 'DT',
 'NN',
 'NN',
 'IN',
 'NN',
 'NN',
 'NNS',
 '.']

###2.4 Build dataset

In [15]:
from torch.utils.data import Dataset

# Finding the maximum length of sentences in the training set
MAX_LEN = max([len(sentence) for sentence in train_sentences])
# print(MAX_LEN)

# Defining a custom Dataset class for part-of-speech tagging
class PosTagging_Dataset(Dataset):
  def __init__(self,
               sentences: List[List[str]],
               tags: List[List[str]],
               tokenizer,
               label2id,
               max_len = MAX_LEN
               ):
    super().__init__()
    self.sentences = sentences
    self.tags = tags
    self.max_len = max_len
    self.tokenizer = tokenizer
    self.label2id = label2id

  # Defining the method to return the length of the dataset
  def __len__(self):
    return len(self.sentences)

  # Defining the method to retrieve a single item from the dataset
  def __getitem__(self,idx):
    input_token = self.sentences[idx]
    label_token = self.tags[idx]

    # Converting input tokens to token IDs using the provided tokenizer
    input_token = self.tokenizer.convert_tokens_to_ids(input_token)

    # attetion mask is required in Bert
    attention_mask = [1] * len(input_token)

    # Converting label tokens to corresponding label IDs using label2id mapping
    labels = [self.label2id[token] for token in label_token]

    # Returning a dictionary containing input_ids, labels, and attention_mask
    return {
        "input_ids": self.pad_and_truncate(input_token, pad_id = self.tokenizer.pad_token_id),
        "labels": self.pad_and_truncate(labels, pad_id=label2id["0"]),
        "attention_mask": self.pad_and_truncate(attention_mask, pad_id = 0)
    }

  # Defining a method to pad or truncate inputs to a fixed length
  def pad_and_truncate(self, inputs:List[int], pad_id: int):
    if len(inputs) < self.max_len:
      padded_inputs = inputs + [pad_id] * (self.max_len - len(inputs))
    else:
      padded_inputs = inputs[: self.max_len]
    return torch.as_tensor(padded_inputs)


###2.5 Dataset loader

In [16]:
# Creating training, validation, test dataset object using the custom PostTagging_Dataset class
train_dataset = PosTagging_Dataset(train_sentences, train_tags, tokenizer, label2id)
val_dataset = PosTagging_Dataset(valid_sentences, valid_tags, tokenizer, label2id)
test_dataset = PosTagging_Dataset(test_sentences, test_tags, tokenizer, label2id)


##3. Train model

In [17]:
# Loading Accuracy from module evaluate
accuracy = evaluate.load("accuracy")

# ignore the label = 0 or padding
ignore_label = label2id["0"]

# Defining a method to compute metrics
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  # Creating mask value, ignore the label = 0 or padding
  mask = labels != ignore_label
  predictions = np.argmax(predictions, axis = -1)
  return accuracy.compute(predictions = predictions[mask], references = labels[mask])


In [18]:
ignore_label

0

In [19]:
label2id["0"]

0

In [20]:
tokenizer.pad_token_id

0

In [21]:
from transformers import TrainingArguments, Trainer
# Initial parameter for training
training_args = TrainingArguments(
    # Directory for log and output
    output_dir = "output_dir",
    # learning rate
    learning_rate = 1e-5,
    # Batch size
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    # Number of epochs
    num_train_epochs = 3,
    # Evaluate after each epoch
    evaluation_strategy = "epoch",
    # Save after each epoch
    save_strategy = "epoch",
    # Save the best output
    load_best_model_at_end = True
)


trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics,
)

In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.047926,0.859302
2,No log,0.041468,0.877748
3,0.157400,0.039414,0.884474


TrainOutput(global_step=516, training_loss=0.15374481423880704, metrics={'train_runtime': 477.8102, 'train_samples_per_second': 17.197, 'train_steps_per_second': 1.08, 'total_flos': 1136892403576908.0, 'train_loss': 0.15374481423880704, 'epoch': 3.0})

##4. Test model

In [23]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.04320026561617851,
 'eval_accuracy': 0.8708445324656953,
 'eval_runtime': 9.713,
 'eval_samples_per_second': 60.538,
 'eval_steps_per_second': 3.809,
 'epoch': 3.0}

##5.Inference

In [24]:
test_sentence = "We are exploring the topic of deep learning"

In [37]:
test_sentence.split()

['We', 'are', 'exploring', 'the', 'topic', 'of', 'deep', 'learning']

In [25]:
input  = torch.as_tensor([tokenizer.convert_tokens_to_ids(test_sentence.split())])

In [26]:
input

tensor([[12865, 10301,   100, 10105, 57680, 10108, 26591, 26901]])

In [27]:
input = input.to("cuda")

In [28]:
outputs = model(input)
outputs.logits.shape

torch.Size([1, 8, 46])

In [29]:
_, preds = torch.max(outputs.logits, -1)
preds = preds[0].cpu().numpy()

In [30]:
pred_tags = ""

for pred in preds:
    pred_tags += id2label[pred] + " "

pred_tags

'O VBP VBG DT NN IN JJ NN '

In [33]:
print(sentence_tags[0])
print(sentences[0])

['NNP', 'NNP', ',', 'CD', 'NNS', 'JJ', ',', 'MD', 'VB', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', 'NNP', 'CD', '.']
['pierre', 'vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov.', '29', '.']


In [38]:
input  = torch.as_tensor([tokenizer.convert_tokens_to_ids(sentences[0])])
input = input.to("cuda")
input

tensor([[35346,   100,   117, 12314, 10855, 12898,   117, 11337, 23707, 10105,
         17936, 10146,   169,   100, 12461,   100, 10386,   119]],
       device='cuda:0')

In [39]:
outputs = model(input)
outputs.logits.shape

torch.Size([1, 18, 46])

In [41]:
_, preds = torch.max(outputs.logits, -1)
preds = preds[0].cpu().numpy()
preds

array([21, 21,  2,  9, 23, 14,  2, 19, 36, 10, 20, 13, 10, 14, 20, 21,  9,
        4])

In [42]:
pred_tags = ""

for pred in preds:
    pred_tags += id2label[pred] + " "

print(pred_tags)
print(sentence_tags[0])


NNP NNP , CD NNS JJ , MD VB DT NN IN DT JJ NN NNP CD . 
['NNP', 'NNP', ',', 'CD', 'NNS', 'JJ', ',', 'MD', 'VB', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', 'NNP', 'CD', '.']
